## Analyse des données public sur l'épidémie de covid19 en France

Source: https://www.data.gouv.fr/en/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/

Données du 29 Décembre

In [78]:
import pandas as pd

In [79]:
df = pd.read_csv('./data/donnees-hospitalieres-classe-age-covid19-2020-12-29-19h03.csv', sep=';',parse_dates=['jour'])

## Analyse temporelle par classe d'âge

Analyse de la dynamique des décès par classe d'àge.

In [80]:
import numpy as np
import pandas_bokeh as pb

In [81]:
pb.output_notebook()
pd.set_option('plotting.backend', 'pandas_bokeh')
# Create Bokeh-Table with DataFrame:
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.models import ColumnDataSource

Loading BokehJS ...

In [82]:
deces_par_age = df[df['cl_age90']>0].pivot_table(index='jour', columns='cl_age90', values='dc',aggfunc='sum')
deces_par_age.plot_bokeh(kind="line",title ="Covid - répartition des décès cumulés",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Décès')

Figure(id='12215', ...)

## Analyse de la situation actuelle

Analyse simplifiée retenant deux groupes d'àge:
* moins de 60 ans
* plus de 60 ans

In [83]:
deces_derniere_situation = df[df['cl_age90']>0].drop(['hosp','rea','rad'], axis=1).set_index(['jour']).last('1D').drop('reg',axis=1).groupby(['jour','cl_age90']).agg('sum').reset_index()

In [84]:

deces_moins_60 = deces_derniere_situation[deces_derniere_situation['cl_age90'] < 69]['dc'].sum()
deces_plus_60 = deces_derniere_situation[deces_derniere_situation['cl_age90'] >= 69]['dc'].sum()

In [85]:
pd.DataFrame(np.array([[deces_moins_60], [deces_plus_60]]),
                   columns=['deces'],index=['moins de 60','plus de 60']).plot_bokeh(kind="pie",title ="Covid - répartition des décès cumulés",
                   figsize =(600,600),xlabel='groupe d\'àge')

__x__values_original


Figure(id='12891', ...)

In [86]:
deces_derniere_situation.drop(['jour'],axis=1).plot_bokeh.bar(
    x='cl_age90',
    y='dc',
    title="Impact du covid19 par catégorie d'àge", 
    alpha=0.6,
    xlabel="catégorie d'àge",
    ylabel="deces" )

Figure(id='13064', ...)

## Analyse comparée avec la démographie française

[https://www.insee.fr/fr/statistiques/4515539?sommaire=4516122]


In [87]:
demo = pd.read_csv('data/BTT_TD_POP1A_2017.CSV',sep=';')
demo_agg = demo.drop(['NIVGEO','CODGEO','LIBGEO','SEXE'],axis=1).groupby('AGEPYR10').agg('sum')


/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Basé sur le dernier recensement

* calcul de la population totale
* calcul de la population des moins de 55 ans
* calcul de la population des plus de 55 ans



In [88]:
nb_total = demo['NB'].sum()
nb_moins_55 = demo_agg.loc[0:55].sum()['NB']
nb_plus_55 = demo_agg.loc[65:].sum()['NB']

In [89]:
print("population total: {:.0f}".format(nb_total))
print("population moins de 55 ans {:.0f} soit un ratio de {:.2f}".format(nb_moins_55,nb_moins_55/nb_total))
print("population plus de 55 ans {:.0f} soit un ratio de {:.2f}".format(nb_plus_55,1-nb_moins_55/nb_total))

population total: 70091267
population moins de 55 ans 56567511 soit un ratio de 0.81
population plus de 55 ans 13523756 soit un ratio de 0.19


In [90]:
res_agg = pd.DataFrame(np.array([[nb_moins_55,deces_moins_60], [nb_plus_55,deces_plus_60]]),
                   columns=['population','deces'],index=['moins de 55/60','plus de 55/60']).astype(int)
stacked_bar = res_agg.plot_bokeh.bar(
    ylabel="population", 
    title="Impact du covid19 par catégorie d'àge", 
    alpha=0.6,
    xlabel='categorie')

### Conclusion

Les plus de 60 ans sont clairement plus à risque: sur 43000 morts à l'hôpital, plus de 41000 ont plus de 60 ans.


## Décès quotidiens par tranche d'âge 

In [91]:
m60_l=[9,19,29,39,49,59]
p60_l=[69,79,89,90]
deces_quo_age = deces_par_age.diff()
deces_quo_age['m60']=deces_quo_age[m60_l].sum(axis=1)
deces_quo_age['p60']=deces_quo_age[p60_l].sum(axis=1)

deces_quo_moy = deces_quo_age.rolling(7).mean()
deces_quo_moy['r60']=deces_quo_moy['p60']/(deces_quo_moy['p60']+deces_quo_moy['m60'])

deces_quo_moy.drop(['m60','p60'],axis=1).plot_bokeh(kind="line",title ="Covid -  décès quotidiens par classe d'âge",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Décès quotidens')

Figure(id='13393', ...)

In [92]:
deces_quo_moy[['m60','p60']].plot_bokeh(kind="line",title ="Covid -  décès quotidiens plus/mois de 60 ans",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Décès quotidens')

Figure(id='14151', ...)

In [93]:
deces_quo_moy[['r60']].plot_bokeh(kind="line",title ="Covid -  décès quotidiens - ratio plus de 60 ans",
                   figsize =(800,800),
                                  ylim=(0,1),
                        xlabel='Date',
                        ylabel='Ratio décès plus de 60 ans')

Figure(id='14471', ...)

## Situation en réanimation

In [94]:
rea_par_age = df[df['cl_age90']>0].pivot_table(index='jour', columns='cl_age90', values='rea',aggfunc='sum')
rea_par_age.plot_bokeh(kind="line",title ="Covid - patients en réanimation",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Patients en réanimation')

Figure(id='14769', ...)

In [95]:
rea_derniere_situation = df[df['cl_age90']>0].drop(['hosp','dc','rad'], axis=1).set_index(['jour']).last('1D').drop('reg',axis=1).groupby(['jour','cl_age90']).agg('sum').reset_index()
rea_moins_60 = rea_derniere_situation[rea_derniere_situation['cl_age90'] < 69]['rea'].sum()
rea_plus_60 = rea_derniere_situation[rea_derniere_situation['cl_age90'] >= 69]['rea'].sum()

In [96]:
rea_moins_60

577

In [97]:
rea_plus_60

2066

In [98]:
rea_moins_60 / (rea_moins_60 + rea_plus_60)

0.21831252364737042

### Conclusion

21.8% des patients en réanimation ont moins de 60 ans au 29 Décembre 2020

In [99]:
rea_par_age['m60'] = rea_par_age[m60_l].sum(axis=1)
rea_par_age['p60'] = rea_par_age[p60_l].sum(axis=1)
rea_par_age[['m60','p60']].plot_bokeh(kind="line",title ="Covid - patients en réanimation plus/moins 60 ans",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Patients en réanimation')


Figure(id='15545', ...)

## Situation à l'hôpital (hors réa)

In [100]:
hosp_par_age = df[df['cl_age90']>0].pivot_table(index='jour', columns='cl_age90', values='hosp',aggfunc='sum')
hosp_par_age.plot_bokeh(kind="line",title ="Covid - patients hospitalisés",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Patients hospitalisés')

Figure(id='15942', ...)

In [101]:
hosp_derniere_situation = df[df['cl_age90']>0].drop(['rea','dc','rad'], axis=1).set_index(['jour']).last('1D').drop('reg',axis=1).groupby(['jour','cl_age90']).agg('sum').reset_index()
hosp_moins_60 = hosp_derniere_situation[hosp_derniere_situation['cl_age90'] < 69]['hosp'].sum()
hosp_plus_60 = hosp_derniere_situation[hosp_derniere_situation['cl_age90'] >= 69]['hosp'].sum()

In [102]:
hosp_plus_60

21739

In [103]:
hosp_moins_60

2825

In [104]:
hosp_moins_60/(hosp_moins_60+hosp_plus_60)

0.11500569939749226

### Conclusion

11.5% des patients hospitalisés ont moins de 60 ans au 29 Décembre 2020

In [105]:
hosp_par_age['p60']=hosp_par_age[p60_l].sum(axis=1)
hosp_par_age['m60']=hosp_par_age[m60_l].sum(axis=1)
hosp_par_age[['m60','p60']].plot_bokeh(kind="line",title ="Covid - patients hospitalisés plus/moins 60 ans",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Patients hospitalisés')

Figure(id='16776', ...)